In [1]:
import numpy as np 
import pandas as pd
import re 
import os

## Loading fthe file updated till the previous month

In [2]:
df1 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\Main.xlsx")

In [3]:
df1.shape

(171136, 17)

### ------------------------- Treating this months data in the current script -----------------------------

In [4]:
## Reading the list of products to be used for search and store operation( This has been made using the file provided by Manish Upreti where the product names were extracted manually )
df = pd.read_csv(r"C:\Users\S1130866\Desktop\Product_Automate\product.csv")
products = [i for i in set(df.Products.str.lower().tolist()) if not isinstance(i, (int, float)) and i != '0']

In [5]:
    map_24d = { v: '2 4 d' for v in ['2 4-d', '24 d', '2,4-d', '2 4 d', '2 4- d', '2 4 -d', '2 4d',
       '2, 4 d', '2, 4-d', '2,4 - d', '2,4 d', '2,4- d', '2,4d', '2-4-d',
       '24d', '2-4 d', '24- d', '2-4- d', '2-4d', '24-d', '2, 4- d',
       '2, 4d', '2- 4-d']}
    
    
    def find_prod(x):
   
        #a = ('azoxystrobin')
        if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample|lab|LAB|LAB USE|lab use|(lab use)|(Lab Use)|(LAB USE)|(Lab Use)',x) :
             return ""
        temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d|_|\.|,|\?|\^|\$|\[|\]|\{|\}',x.lower())]))
        if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
             temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
             #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
              #temp_products += a
        if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
             if len(temp_products)<1:
                    temp_products += 'fertilizer'
        if len(temp_products)>1 :
            temp_products = [i for i in temp_products if i!='pesticide']
            temp_products = [i for i in temp_products if i!='pesticides']
            temp_products = [i for i in temp_products if i!='rynaxypyr']
        return temp_products
        # Creating a function for the same
        
    def suf(x):
        temp_suffix = [] 
        if re.search('62% IPA SALT|62% ipa salt',x):
            temp_suffix = "62% IPA SALT"
        elif re.search('IPA SALT|ipa salt',x):
            temp_suffix = "IPA SALT"
        elif re.search('tech.{,50}|TECH.{,50}',x):
            temp_suffix = "TECH"
        elif re.search('41% SL|41% sl',x):
            temp_suffix = '41% SL'
        elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
            temp_suffix = "ALION PLUS"
        else :
            temp_suffix = "Others"
        return temp_suffix    
    
    

In [6]:
cm_exp = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\September 2023\Export\38_Exp_TY2_Sep-23.xlsx")
cm_imp = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\September 2023\Import\38_Imp_TY2_Sep-23.xlsx")

###   ------------ EXPORT FILE --------------------

In [7]:



    ##Mapping the Product Description column as string as sometimes it may not be read as a string
    cm_exp['Goods Description'] = cm_exp['Goods Description'].map(str)
    
    print("Working on Product Mapping")
    temp = cm_exp['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

    temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
    print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
    cm_exp.loc[np.logical_and(cm_exp['Unit'].gt('GMS'), np.less_equal(cm_exp['Quantity'], 1)), 'Product'] = ''
    cm_exp.loc[cm_exp["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
    temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
    temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
    temp_processed.replace('mancozebe','mancozeb',inplace = True)
    temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
    temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
    temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
    temp_processed.replace('glyphosatee','glyphosate',inplace = True)
    temp_processed.replace('','Others',inplace = True)
    
    cm_exp['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
    cm_exp['Product'] = cm_exp['Product'].fillna('Others')
    
    print("Product Mapping done , working on Suffix creation")
    
  
    
    cm_exp['suffix'] =  cm_exp['Goods Description'].apply(lambda x: suf(x))
    print("Suffix Creation done")
 


Working on Product Mapping
Product Mapping done , working on other conditions
Product Mapping done , working on Suffix creation
Suffix Creation done


In [8]:
## Bringing the Export file in the required format : 

### Block for altering the entire data into the required format

temp2 = cm_exp.drop(['Declaration No', 'HS Four Digit', 'HS Code','Destination Port','Indian Port','Address', 'City','Consinee Address','Port Code', 'IEC Phone', 'IEC Email', 'IEC Contact'],axis = 1)
temp2['Product Name'] = temp2['Product'] + " " + temp2['suffix']
temp2 = temp2.drop('suffix',axis = 1)
temp2['Type'] = 'Export'
temp2.rename(columns = {'Exporter Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Value Usd':'UNIT PRICE (USD)','Fob Usd':'Total_VALUE_USD','Currency':'INVOICE CURRENCY','Consinee Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)

temp2 = temp2.loc[:,["Type","Date","IMPORTER/EXPORTER NAME","Country","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Product Name"]]
temp2['Product Name'].replace('Others Others','Others',inplace = True)

### -------IMPORT FILE ------

In [9]:

    ##Mapping the Product Description column as string as sometimes it may not be read as a string
    cm_imp['Goods Description'] = cm_imp['Goods Description'].map(str)
    
    print("Working on Product Mapping")
    temp = cm_imp['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

    temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
    print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
    cm_imp.loc[np.logical_and(cm_imp['Unit'].gt('GMS'), np.less_equal(cm_imp['Quantity'], 1)), 'Product'] = ''
    cm_imp.loc[cm_imp["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
    temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
    temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
    temp_processed.replace('mancozebe','mancozeb',inplace = True)
    temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
    temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
    temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
    temp_processed.replace('glyphosatee','glyphosate',inplace = True)
    temp_processed.replace('','Others',inplace = True)
    
    cm_imp['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
    cm_imp['Product'] = cm_imp['Product'].fillna('Others')
    
    print("Product Mapping done , working on Suffix creation")
    
  
    
    cm_imp['suffix'] =  cm_imp['Goods Description'].apply(lambda x: suf(x))
    print("Suffix Creation done")
 

Working on Product Mapping
Product Mapping done , working on other conditions
Product Mapping done , working on Suffix creation
Suffix Creation done


In [10]:
## Bringing the Import file in the required format : 

### Block for altering the entire data into the required format


temp = cm_imp.drop(['Port Code', 'Indian Port', 'Declaration No','Address', 'City', 'CHA No', 'CHA Name','HS Code','Total_Duty_Usd', 'Loading Port','Supplier Address', 'BE Type', 'IEC Phone','IEC Email', 'IEC Contact'],axis = 1)
temp['Product Name'] = temp['Product'] + " " + temp['suffix']
temp = temp.drop('suffix',axis = 1)
temp['Type'] = 'Import'
temp.rename(columns = {'Importer Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Price Usd':'UNIT PRICE (USD)','Total Value Usd':'Total_VALUE_USD','Inv Currency':'INVOICE CURRENCY','Supplier Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)



temp = temp.loc[:,["Type","Date","IMPORTER/EXPORTER NAME","Country","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Product Name"]]
temp['Product Name'].replace('Others Others','Others',inplace = True)

In [11]:
data = pd.concat([temp,temp2])

In [12]:
data2 = data[data['Molecule Name'] != 'Others']

In [13]:
#### -------------------------------  WORKING ON THE COUNTRY, COMPANY   -------------------------



#### Working on the compnay column i,e IMPORTER/EXPORTER NAME

company = pd.read_excel(r'C:\Users\S1130866\Desktop\Product_Automate\Country and company.xlsx')
country = pd.read_excel(r'C:\Users\S1130866\Desktop\Product_Automate\Country and company.xlsx',sheet_name = "Country") 

unique = company.drop_duplicates()
unique_mismatches = unique[unique['Company'].str.lower() != unique['New'].str.lower()]
company_key_value = dict(zip(unique_mismatches['Company'].str.lower(),unique_mismatches['New'].str.lower()))
data2['IMPORTER/EXPORTER NAME'] = data2['IMPORTER/EXPORTER NAME'].str.lower().replace(company_key_value)



### Working on the country column



unique2 = country.drop_duplicates()
unique_mismatches2 = unique2[unique2['Country'].str.lower() != unique2['New'].str.lower()]
country_key_value = dict(zip(unique_mismatches2['Country'].str.lower(),unique_mismatches2['New'].str.lower()))
data2['Country'] = data2['Country'].str.lower().replace(country_key_value)







C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/1478236967.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['IMPORTER/EXPORTER NAME'] = data2['IMPORTER/EXPORTER NAME'].str.lower().replace(company_key_value)
C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/1478236967.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Country'] = data2['Country'].str.lower().replace(country_key_value)


In [14]:
### -------------------------------  UNIT COLUMN -------------------------------------------



data2['UNIT'] = data2['UNIT'].apply(lambda x: x.strip())




#1l = 1kg

C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/4267606557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['UNIT'] = data2['UNIT'].apply(lambda x: x.strip())


In [15]:

## Creating a Quantity_MT column similar to the QTY Column

data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'KGS' else x['QTY'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'LTR' else x['QTY'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'LT/KG' else x['QTY'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 1 if x['UNIT']== 'MTS' else x['QTY'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 1 if x['UNIT']== 'MLT' else x['QTY'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.000001 if x['UNIT']== 'MGS' else x['QTY'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.000001 if x['UNIT']== 'GMS' else x['QTY'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.000453592 if x['UNIT']== 'LBS' else x['QTY'], axis=1)




C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/1343133519.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'KGS' else x['QTY'], axis=1)
C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/1343133519.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['QTY_MT'] = data2.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'LTR' else x['QTY'], axis=1)
C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/1343133519.py:5: SettingW

In [16]:

## Values of the units to be converted into the required format 

"""

'KGS':'KG/LT',
'KGS':'KG/LT ,
'GMS':'GMs/ML' ,
'GMS':'GMs/ML' ,
'LT/KG':'KG/LT'
'LTR':'KG/LT',
'LTR':'KG/LT',
'MTS':'Metric Tons' ,
'MTS':'Metric Tons' ,
'MLT':'Metric LT '
'MGS':'GMs/ML' ,
'MGS':'GMs/ML'


"""



unit_dict_Other = {
'BAG':'Other',
'BTL':'Other', 
'CAN':'Other',
'CBM':'Other',
'CTM':'Other',
'DOZ':'Other',
'DRM':'Other',
'GRS':'Other',
'KGA':'Other',
'KIT':'Other',
'KLR':'Other',
'MTR':'Other',
'MTR':'Other',
'NOS':'Other',
'NOS':'Other',
'PAC':'Other',
'PCS':'Other',
'PCS':'Other',
'PRS':'Other',
'RLS':'Other',
'SET':'Other',
'SET':'Other',
'SQM':'Other',
'THD':'Other',
'TUB':'Other',
'UNT':'Other',
'UNT':'Other',
'VLS':'Other',
'VLS':'Other',
'TBS':'Other',
'BRL':'Other',
'KGS':'MT',
'GMS':'MT',
'LT/KG':'MT',
'LTR':'MT',
'MTS':'MT',
'MLT':'MT',
'MGS':'MT',
'LBS':'MT',
'MT':'MT',
'Other':'Other'
}




data2['UNIT_MT'] = data2['UNIT'].map(unit_dict_Other)


C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/1708701055.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['UNIT_MT'] = data2['UNIT'].map(unit_dict_Other)


In [17]:
###  ----------- ORDER FOR AI's  are sorted here--------------------

AI_Order = pd.read_excel(r"C:\Users\S1130866\Desktop\Product_Automate\Tech vs Other.xlsx")


AI_Mapping = dict(zip(AI_Order['Product Name 1'].str.lower(),AI_Order['Product Name 2'].str.lower()))

AI_Mapping

def AI_MAP(x) :
    a = 'Others'
    if x in AI_Mapping.keys():
        x.map(AI_Mapping)
    else  : 
        elements =  [i for i in x.split(', ')]
        if len(elements)>1:
            return str(a)
        else : 
            return x

data2['AI'] = data2['Product Name'].apply(lambda x : x.lower())

data2['AI'] = data2['AI'].map(AI_Mapping).fillna(data2['Product Name'])

data2['Country'] = data2['Country'].str.capitalize()
data2['IMPORTER/EXPORTER NAME'] = data2['IMPORTER/EXPORTER NAME'].str.capitalize()






C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/3857703034.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['AI'] = data2['Product Name'].apply(lambda x : x.lower())
C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/3857703034.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['AI'] = data2['AI'].map(AI_Mapping).fillna(data2['Product Name'])
C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/3857703034.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [18]:
### Converting the QTY_MT unit into Metric tonnes

data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.001 if x['UNIT']== 'KGS' else x['QTY_MT'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.001 if x['UNIT']== 'LTR' else x['QTY_MT'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.001 if x['UNIT']== 'LT/KG' else x['QTY_MT'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 1 if x['UNIT']== 'MTS' else x['QTY_MT'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 1 if x['UNIT']== 'MLT' else x['QTY_MT'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.000001 if x['UNIT']== 'MGS' else x['QTY_MT'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.000001 if x['UNIT']== 'GMS' else x['QTY_MT'], axis=1)
data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.000453592 if x['UNIT']== 'LBS' else x['QTY_MT'], axis=1)



C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/88655050.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.001 if x['UNIT']== 'KGS' else x['QTY_MT'], axis=1)
C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/88655050.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['QTY_MT'] = data2.apply(lambda x: x['QTY_MT'] * 0.001 if x['UNIT']== 'LTR' else x['QTY_MT'], axis=1)
C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/88655050.py:5: Se

In [19]:

## Makign changes to the supplier Buyer column by replacing 'to the order'

data2 = data2.loc[data2['QTY_MT']>0.01]

data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To the order of','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To order','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To the order','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To the order of g','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To the order o','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To order of','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To the order of the ','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To order .','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To th eorder','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To order o','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('Totheorder of','')
data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].replace('','Others')


data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.lower().replace(company_key_value)



data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.capitalize()

C:\Users\S1130866\AppData\Local\Temp/ipykernel_19396/2017787190.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].str.replace('To order .','')


In [ ]:

data2['SUPPLIER/BUYER NAME'] = data2['SUPPLIER/BUYER NAME'].fillna('')
data2 = data2.reset_index()
data2.loc[data2['IMPORTER/EXPORTER NAME'].str.lower().str.startswith('bay'),'AI'] = 'Bayer -' + data2['AI']

In [20]:
### Merging the 2 datasets together

df_compiled = pd.concat([df1,data2],axis = 0 )

In [21]:
## Dealing with the average unit price 

unit_dict = {'KGS':1000,'LTR':1000,'MTS':1,'LT/KG':1000,'MLT':1000000,'GMS':1000000,'MTS':1}


# Define lambda function to multiply Unit and Unit Price columns
multiply_func = lambda row: row['UNIT PRICE (USD)'] * unit_dict.get(row['UNIT'], 1)

# Create new column 'Total Price' using the apply() method with lambda function
df_compiled['Unit_Price_MT'] = df_compiled.apply(multiply_func, axis=1)



In [22]:
#######
#Managing the Bayer and 62% IPA SALT condition

# Create sample dataframe


# Update values in existing column based on conditions
data2.loc[(data2['IMPORTER/EXPORTER NAME'] == 'Bayer') & (data2['PRODUCT DESCRIPTION'].str.contains('62')), 'AI'] = 'glyphosate 62% ipa salt'



In [23]:
import regex as re

In [24]:
df_compiled['AI'] = df_compiled['AI'].str.replace('metolachlor tech','s-metolachlor tech')


df_compiled['AI'] = df_compiled['AI'].apply(lambda x : re.sub(r'\bmetolachlor tech\b','s-metolachlor tech',x))

In [26]:
df_compiled.to_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\Main.xlsx",sheet_name = 'Data',index = False)

In [37]:
df_compiled['SUPPLIER/BUYER NAME'] = df_compiled['SUPPLIER/BUYER NAME'].fillna('')

#b_columns = df_compiled[df_compiled['SUPPLIER/BUYER NAME'].str.lower().str.startswith('bay')]['SUPPLIER/BUYER NAME'].unique()
df_compiled = df_compiled.reset_index()
df_compiled.loc[df_compiled['IMPORTER/EXPORTER NAME'].str.lower().str.startswith('bay'),'AI'] = 'Bayer -' + df_compiled['AI']

In [38]:
df_compiled.head()

,index,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,QTY_MT,UNIT_MT,AI,Unit_Price_MT
0,0,Import,2022-01-01,S s international,Indonesia,GUM ROSIN WW GRADE IN ZINC DRUMS (NOT FOR USE ...,19200.0,KGS,1.735000,3.331200e+04,USD,Perum perhutani,zinc,zinc Others,19.2,MT,not relevant,1735.000000
1,1,Import,2022-01-01,Dbl ventures private limited,Malaysia,COATED CALCIUM CARBONATE PRI-2T (25KG/PRINTED ...,185.5,MTS,161.528847,2.996360e+04,USD,Pulai rock industries sdn bhd,calcium,calcium Others,185.5,MT,calcium others,161.528847
2,2,Import,2022-01-01,Dbl ventures private limited,Malaysia,COATED CALCIUM CARBONATE PRI-2T (25KG/PRINTED ...,265.0,MTS,180.397302,4.780529e+04,USD,Pulai rock industries sdn bhd,calcium,calcium Others,265.0,MT,calcium others,180.397302
3,3,Import,2022-01-01,Merchem limited,China,INSOLUBLE SULPHUR HD OT-20,14000.0,KGS,1.600000,2.240000e+04,USD,Shandong yanggu huatai international,sulphur,sulphur Others,14.0,MT,not relevant,1600.000000
4,4,Import,2022-01-01,Bayer,Japan,FLUBENDIAMIDE (NIH-KM) TC 1X500KG FBC WW (CIB ...,10000.0,KGS,188.022294,1.880223e+06,INR,Bayer ag,flubendiamide,flubendiamide Others,10.0,MT,Bayer -flubendiamide others,188022.294232


In [39]:
df_compiled.to_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\Main.xlsx",sheet_name = 'Data',index = False)